# Guidelines for ETL Project

This document contains guidelines, requirements, and suggestions for Project 1.

## Project Proposal

Before you start writing any code, remember that you only have one week to complete this project. View this project as a typical assignment from work. Imagine a bunch of data came in and you and your team are tasked with migrating it to a production data base.

Take advantage of your Instructor and TA support during office hours and class project work time. They are a valuable resource and can help you stay on track.

## Finding Data

Your project must use 2 or more sources of data. We recommend the following sites to use as sources of data:

* [data.world](https://data.world/)

* [Kaggle](https://www.kaggle.com/)

You can also use APIs or data scraped from the web. However, get approval from your instructor first. Again, there is only a week to complete this!

## Data Cleanup & Analysis

Once you have identified your datasets, perform ETL on the data. Make sure to plan and document the following:

* The sources of data that you will extract from.

* The type of transformation needed for this data (cleaning, joining, filtering, aggregating, etc).

* The type of final production database to load the data into (relational or non-relational).

* The final tables or collections that will be used in the production database.

You will be required to submit a final technical report with the above information and steps required to reproduce your ETL process.

## Project Report

At the end of the week, your team will submit a Final Report that describes the following:

* **E**xtract: your original data sources and how the data was formatted (CSV, JSON, MySQL, etc).

* **T**ransform: what data cleaning or transformation was required.

* **L**oad: the final database, tables/collections, and why this was chosen.

Please upload the report to Github and submit a link to Bootcampspot.

- - -

### ETL  

In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
# Store fundamentals CSV into DataFrame
financial_csv_file = "./Resources/nyse-fundamentals.csv"
financial_data_df = pd.read_csv(financial_csv_file)
financial_data_df.head()

,Unnamed: 0,Ticker Symbol,Period Ending,Accounts Payable,Accounts Receivable,Add'l income/expense items,After Tax ROE,Capital Expenditures,Capital Surplus,Cash Ratio,...,Total Current Assets,Total Current Liabilities,Total Equity,Total Liabilities,Total Liabilities & Equity,Total Revenue,Treasury Stock,For Year,Earnings Per Share,Estimated Shares Outstanding
0,0,AAL,2012-12-31,3.068000e+09,-222000000.0,-1.961000e+09,23.0,-1.888000e+09,4.695000e+09,53.0,...,7.072000e+09,9.011000e+09,-7.987000e+09,2.489100e+10,1.690400e+10,2.485500e+10,-367000000.0,2012.0,-5.60,3.350000e+08
1,1,AAL,2013-12-31,4.975000e+09,-93000000.0,-2.723000e+09,67.0,-3.114000e+09,1.059200e+10,75.0,...,1.432300e+10,1.380600e+10,-2.731000e+09,4.500900e+10,4.227800e+10,2.674300e+10,0.0,2013.0,-11.25,1.630222e+08
2,2,AAL,2014-12-31,4.668000e+09,-160000000.0,-1.500000e+08,143.0,-5.311000e+09,1.513500e+10,60.0,...,1.175000e+10,1.340400e+10,2.021000e+09,4.120400e+10,4.322500e+10,4.265000e+10,0.0,2014.0,4.02,7.169154e+08
3,3,AAL,2015-12-31,5.102000e+09,352000000.0,-7.080000e+08,135.0,-6.151000e+09,1.159100e+10,51.0,...,9.985000e+09,1.360500e+10,5.635000e+09,4.278000e+10,4.841500e+10,4.099000e+10,0.0,2015.0,11.39,6.681299e+08
4,4,AAP,2012-12-29,2.409453e+09,-89482000.0,6.000000e+05,32.0,-2.711820e+08,5.202150e+08,23.0,...,3.184200e+09,2.559638e+09,1.210694e+09,3.403120e+09,4.613814e+09,6.205003e+09,-27095000.0,2012.0,5.29,7.328355e+07


In [3]:
# Store securities CSV into DataFrame
securities_csv_file = "./Resources/nyse-securities.csv"
securities_data_df = pd.read_csv(securities_csv_file)
securities_data_df.head()

,Ticker symbol,Security,SEC filings,GICS Sector,GICS Sub Industry,Address of Headquarters,Date first added,CIK
0,MMM,3M Company,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",NaN,66740
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800
2,ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152
3,ACN,Accenture plc,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373
4,ATVI,Activision Blizzard,reports,Information Technology,Home Entertainment Software,"Santa Monica, California",2015-08-31,718877


### Cleaning the dataframes (selecting and renaming columns)

In [4]:
# Create new data with select columns
new_financial_data_df = financial_data_df[['Ticker Symbol', 'Period Ending', 'Accounts Payable', 'Accounts Receivable', 'Cost of Revenue', 'Inventory', 'Net Income', 'Sales, General and Admin.', 'Total Revenue']].copy()
new_financial_data_df.head()

# Clean DataFrame by renaming columns and setting ticker to index
financials_transformed = new_financial_data_df.rename(columns={"Ticker Symbol": "ticker",
                                                            "Period Ending": "year",
                                                            "Accounts Payable": "accounts_payable",
                                                            "Accounts Receivable": "accounts_receivable",
                                                            "Cost of Revenue": "cost_of_rev",
                                                            "Inventory": "inventory",
                                                            "Net Income": "net_income",
                                                            "Sales, General and Admin.": "sga",
                                                            "Total Revenue": "total_revenue"})

# Transform date to year
financials_transformed['year'] = pd.to_datetime(financials_transformed['year'])
financials_transformed['year'] = financials_transformed['year'].dt.year


financials_transformed.head()

,ticker,year,accounts_payable,accounts_receivable,cost_of_rev,inventory,net_income,sga,total_revenue
0,AAL,2012,3.068000e+09,-222000000.0,1.049900e+10,5.800000e+08,-1.876000e+09,1.297700e+10,2.485500e+10
1,AAL,2013,4.975000e+09,-93000000.0,1.101900e+10,1.012000e+09,-1.834000e+09,1.291300e+10,2.674300e+10
2,AAL,2014,4.668000e+09,-160000000.0,1.562000e+10,1.004000e+09,2.882000e+09,2.068600e+10,4.265000e+10
3,AAL,2015,5.102000e+09,352000000.0,1.109600e+10,8.630000e+08,7.610000e+09,2.127500e+10,4.099000e+10
4,AAP,2012,2.409453e+09,-89482000.0,3.106967e+09,2.308609e+09,3.876700e+08,2.440721e+09,6.205003e+09


In [6]:
# Add percentage of SGA to Total Revenue
financials_transformed['sga_percentage'] = financials_transformed['sga'] / financials_transformed['total_revenue']
financials_transformed.head()

,ticker,year,accounts_payable,accounts_receivable,cost_of_rev,inventory,net_income,sga,total_revenue,sga_percentage
0,AAL,2012,3.068000e+09,-222000000.0,1.049900e+10,5.800000e+08,-1.876000e+09,1.297700e+10,2.485500e+10,0.522108
1,AAL,2013,4.975000e+09,-93000000.0,1.101900e+10,1.012000e+09,-1.834000e+09,1.291300e+10,2.674300e+10,0.482855
2,AAL,2014,4.668000e+09,-160000000.0,1.562000e+10,1.004000e+09,2.882000e+09,2.068600e+10,4.265000e+10,0.485018
3,AAL,2015,5.102000e+09,352000000.0,1.109600e+10,8.630000e+08,7.610000e+09,2.127500e+10,4.099000e+10,0.519029
4,AAP,2012,2.409453e+09,-89482000.0,3.106967e+09,2.308609e+09,3.876700e+08,2.440721e+09,6.205003e+09,0.393347


In [7]:
# Create new data with select columns
new_securities_data_df = securities_data_df[['Ticker symbol', 'Security', 'GICS Sector', 'GICS Sub Industry']].copy()
new_securities_data_df.head()

# Clean DataFrame by renaming columns and setting ticker to index
securities_transformed = new_securities_data_df.rename(columns={"Ticker symbol": "ticker",
                                                          "Security": "security_name",
                                                          "GICS Sector": "gics_sector",
                                                          "GICS Sub Industry": "gics_sub_industry"})
securities_transformed.head()

,ticker,security_name,gics_sector,gics_sub_industry
0,MMM,3M Company,Industrials,Industrial Conglomerates
1,ABT,Abbott Laboratories,Health Care,Health Care Equipment
2,ABBV,AbbVie,Health Care,Pharmaceuticals
3,ACN,Accenture plc,Information Technology,IT Consulting & Other Services
4,ATVI,Activision Blizzard,Information Technology,Home Entertainment Software


### Connecting to database

In [8]:
# Connect to local database
connection_string = "root:<password>@localhost/company_db"
engine = create_engine(f'mysql://{connection_string}')

In [9]:
# Check for tables
engine.table_names()

['company_fundamentals', 'company_securities']

In [ ]:
# Use pandas to load csv converted DataFrame into database

In [11]:
financials_transformed.to_sql(name='company_fundamentals', con=engine, if_exists='append', index=False)

In [ ]:
# Use pandas to load json converted DataFrame into database

In [12]:
securities_transformed.to_sql(name='company_securities', con=engine, if_exists='append', index=False)

In [ ]:
# Confirm data has been added by querying the company_fundamentals table

In [13]:
pd.read_sql_query('select * from company_fundamentals', con=engine).head()

,id,ticker,year,accounts_payable,accounts_receivable,cost_of_rev,inventory,net_income,sga,total_revenue,sga_percentage
0,1,AAL,2012,3.068000e+09,-222000000.0,1.049900e+10,5.800000e+08,-1.876000e+09,1.297700e+10,2.485500e+10,0.52
1,2,AAL,2013,4.975000e+09,-93000000.0,1.101900e+10,1.012000e+09,-1.834000e+09,1.291300e+10,2.674300e+10,0.48
2,3,AAL,2014,4.668000e+09,-160000000.0,1.562000e+10,1.004000e+09,2.882000e+09,2.068600e+10,4.265000e+10,0.49
3,4,AAL,2015,5.102000e+09,352000000.0,1.109600e+10,8.630000e+08,7.610000e+09,2.127500e+10,4.099000e+10,0.52
4,5,AAP,2012,2.409453e+09,-89482000.0,3.106967e+09,2.308609e+09,3.876700e+08,2.440721e+09,6.205003e+09,0.39


In [ ]:
# Confirm data has been added by querying the company_securities table

In [14]:
pd.read_sql_query('select * from company_securities', con=engine).head()

,id,ticker,security_name,gics_sector,gics_sub_industry
0,1,MMM,3M Company,Industrials,Industrial Conglomerates
1,2,ABT,Abbott Laboratories,Health Care,Health Care Equipment
2,3,ABBV,AbbVie,Health Care,Pharmaceuticals
3,4,ACN,Accenture plc,Information Technology,IT Consulting & Other Services
4,5,ATVI,Activision Blizzard,Information Technology,Home Entertainment Software
